## Adding convolutional layers

In this step we will switch from plain neural networks to convolutional neural networks and see if this makes any difference.

In [17]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

import numpy as np
import math

# fixed random seed to have consistent results
np.random.seed(123)

# dimensions of our images.
img_width, img_height = 300, 300

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 150

In [18]:
input_shape = (img_height, img_width, 3)

In [19]:
model = Sequential()

# replacing Dense layer with convolution layer
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape=input_shape)) 
# Think of max pooling as resizing the image to half its dimesion
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 156800)            0         
__________

Notice that number of trainable parameters decreased from 8 millions to 2.6 millions.

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [23]:
model.fit_generator(
    train_generator,
    steps_per_epoch= math.ceil(nb_train_samples / batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= math.ceil(nb_validation_samples/batch_size))

Epoch 1/5
20/20 [==============================] - 62s 3s/step - loss: 0.7366 - acc: 0.5250 - val_loss: 0.6721 - val_acc: 0.5733
Epoch 2/5
20/20 [==============================] - 60s 3s/step - loss: 0.6066 - acc: 0.6653 - val_loss: 0.5494 - val_acc: 0.7533
Epoch 3/5
20/20 [==============================] - 61s 3s/step - loss: 0.5427 - acc: 0.7330 - val_loss: 0.5699 - val_acc: 0.6833
Epoch 4/5
20/20 [==============================] - 61s 3s/step - loss: 0.4961 - acc: 0.7647 - val_loss: 0.5109 - val_acc: 0.7433
Epoch 5/5
20/20 [==============================] - 61s 3s/step - loss: 0.4915 - acc: 0.7630 - val_loss: 0.5129 - val_acc: 0.7633


Not bad for just three convolution layers, more than 70% now.

Let's do something different in notebook #5 : Transfer Learning.